In [1]:
import sys
sys.path.append("../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import torch


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)
training_data['x'] = torch.tensor(training_data['x']).float()
training_data['dx'] = torch.tensor(training_data['dx']).float()
training_data['t'] = torch.tensor(training_data['t']).float()
training_data['z'] = torch.tensor(training_data['z']).float()
training_data['dz'] = torch.tensor(training_data['dz']).float()


validation_data['x'] = torch.tensor(validation_data['x']).float()
validation_data['dx'] = torch.tensor(validation_data['dx']).float()
validation_data['t'] = torch.tensor(validation_data['t']).float()
validation_data['z'] = torch.tensor(validation_data['z']).float()
validation_data['dz'] = torch.tensor(validation_data['dz']).float()


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]
print(params['widths'][::-1])
# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 1

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

[32, 64]


# Run training experiments

In [4]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")
    
    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0


c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32),
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:165: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32)


TRAINING
Epoch 0


c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_total_loss, train_losses, _ = network.define_loss( torch.tensor(train_dict['x'], dtype=torch.float32) , torch.tensor(train_dict['dx'], dtype=torch.float32), params=params)
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_total_loss, val_losses, _ = network.define_loss( torch.tensor(validation_dict['x'], dtype=torch.float32), torch.tensor(validation_

Epoch 0
   Training Total Loss: 0.03638455644249916
   Training decoder Loss: 0.03519994020462036
   Training sindy_z Loss: 182.57476806640625
   Training sindy_x Loss: 11.764410018920898
   Training sindy_regularization Loss: 0.8177505135536194
   Validation Total Loss: 0.044518351554870605
   Validation decoder Loss: 0.04307567700743675
   Validation sindy_z Loss: 194.98289489746094
   Validation sindy_x Loss: 14.344962120056152
   Validation sindy_regularization Loss: 0.8177505135536194
Decoder Loss Ratio: 0.221707, Decoder SINDy Loss Ratio: 0.999057
Epoch 1
Epoch 1
   Training Total Loss: 0.028193293139338493
   Training decoder Loss: 0.027033640071749687
   Training sindy_z Loss: 119.98314666748047
   Training sindy_x Loss: 11.52031135559082
   Training sindy_regularization Loss: 0.7621705532073975
   Validation Total Loss: 0.03528674319386482
   Validation decoder Loss: 0.033872924745082855
   Validation sindy_z Loss: 140.07217407226562
   Validation sindy_x Loss: 14.061970710754